*Импортируем библиотеки*

In [1]:
from sqlalchemy import create_engine

import pandas as pd
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

*Создаём подключение к БД*

In [2]:
def connection_yandex_cloud_demo(echo):
    """Connection to DataBase dwh"""
    login = 'student'
    password = 'student!'
    host = 'rc1b-7ng6ih3jte3824x8.mdb.yandexcloud.net'
    port = '6432'
    db_name = 'demo'
    return create_engine(f'postgresql://{login}:{password}@{host}:{port}/{db_name}', echo=echo)

engine_demo = connection_yandex_cloud_demo(echo=True)

*Распаковываем и загружаем данные*

In [ ]:
# import zipfile
# with zipfile.ZipFile('data/fraud.csv.zip', 'r') as archive:
#     archive.extractall('data/')

In [4]:
!ls

HW_7.docx
HW_7.pdf
Lesson_7 (CREATE, ALTER, DROP) CREATE).ipynb
SQL_Lesson_7.pdf
SQL_Lesson_7.pptx
data
~$SQL_Lesson_7.pptx


In [3]:
df = pd.read_csv('data/da_data_n1-2.csv', sep=';')

*Посмотрим на датасет*

In [5]:
df.shape

(126497, 4)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126497 entries, 0 to 126496
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   product  126497 non-null  object
 1   year     126497 non-null  int64 
 2   month    126497 non-null  object
 3   value    126497 non-null  object
dtypes: int64(1), object(3)
memory usage: 3.9+ MB


In [23]:
df.head(5)

,product,year,month,value
0,"Фарш мясной, кг",2003,январь,"96,59"
1,"Фарш мясной, кг",2003,февраль,"96,72"
2,"Фарш мясной, кг",2003,март,"97,02"
3,"Фарш мясной, кг",2003,апрель,"97,09"
4,"Фарш мясной, кг",2003,май,"97,36"


In [19]:
# df

*Посмотрим на пропуски*

In [10]:
df.isna().sum()

product    0
year       0
month      0
value      0
dtype: int64

*Определим первичный ключ*

In [15]:
temp_df = df.groupby(['product', 'year', 'month']).count()

In [17]:
temp_df[temp_df.value > 1]

,,,value
product,year,month,


*Загрузим данные*

In [20]:
?df.to_sql

In [22]:
df.to_sql(name='goods'
         ,con=engine_demo
         ,schema='public'
         ,if_exists='replace'
         ,index=True)

2024-01-30 21:17:51,774 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2024-01-30 21:17:51,775 INFO sqlalchemy.engine.Engine [cached since 113.1s ago] {'schema': 'public', 'name': 'goods'}
2024-01-30 21:17:51,842 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2024-01-30 21:17:51,843 INFO sqlalchemy.engine.Engine [cached since 113.1s ago] {'schema': 'public', 'name': 'goods'}
2024-01-30 21:17:51,896 INFO sqlalchemy.engine.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind in ('r', 'p')
2024-01-30 21:17:51,897 INFO sqlalchemy.engine.Engine [cached since 113.1s ago] {'schema': 'public'}
2024-01-30 21:17:51,930 INFO sqlalchemy.engine.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_class c


2024-01-30 21:17:52,108 INFO sqlalchemy.engine.Engine [cached since 113.1s ago] {'table_oid': 27022}
2024-01-30 21:17:52,122 INFO sqlalchemy.engine.Engine 
            SELECT
                pgd.description as table_comment
            FROM
                pg_catalog.pg_description pgd
            WHERE
                pgd.objsubid = 0 AND
                pgd.objoid = %(table_oid)s
        
2024-01-30 21:17:52,123 INFO sqlalchemy.engine.Engine [cached since 113.1s ago] {'table_oid': 27022}
2024-01-30 21:17:52,162 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-30 21:17:52,164 INFO sqlalchemy.engine.Engine 
DROP TABLE public.goods
2024-01-30 21:17:52,165 INFO sqlalchemy.engine.Engine [no key 0.00149s] {}
2024-01-30 21:17:52,203 INFO sqlalchemy.engine.Engine COMMIT
2024-01-30 21:17:52,231 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-30 21:17:52,234 INFO sqlalchemy.engine.Engine 
CREATE TABLE public.goods (
	index BIGINT, 
	product TEXT, 
	year BIGINT, 
	month TEXT, 
	val

497

In [ ]:
df.shape[0]

In [ ]:
engine.execute('ALTER TABLE ... ')

На основе загруженных данных создадим представление:
- Наименование товара
- Текущая стоимость товара (последняя из имеющихся данных)
- Минимальная стоимость товара
- Среднегодовая стоимость товара
- Среднемесячная стоимость товара
- Макисмальная стоимость товара